***<h1><b>Applied Machine Learning<b></h1>***
<h2>Αλέξανδρος - Ιωάννης Δουνάκης</h2> 
<h3>3170044</h3>




<h5>Εργασία 1 </h5>

<h6><b>Q1</b></h6>
<p>Obtaining the Data</p>

In [126]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

raw_data = pd.read_csv("2757408.csv",parse_dates=["DATE"] )
raw_data.set_index('DATE' , inplace=True)

print(raw_data.loc[ (raw_data.index > '2010-1-1') & (raw_data.index < '2019-12-31')])

raw_data2 = pd.read_csv("athens.csv",parse_dates=["DATE"])
raw_data2.set_index('DATE', inplace=True)

print(raw_data2.loc[ (raw_data2.index > '2010-1-1') & (raw_data2.index < '2019-12-31')])


# (raw_data['TAVG'] == 0.0).value_counts()

# np.where(raw_data['TAVG'] == 0 )[0].sum()
# raw_data.to_numpy()
# raw_data['TAVG'].value_counts(ascending=True)

                STATION            NAME  PRCP  TAVG  TMAX  TMIN
DATE                                                           
2010-01-02  GR000016716  HELLINIKON, GR  0.00  61.0  64.0  57.0
2010-01-03  GR000016716  HELLINIKON, GR  0.00  56.0  60.0  51.0
2010-01-04  GR000016716  HELLINIKON, GR  0.00  48.0  52.0  42.0
2010-01-05  GR000016716  HELLINIKON, GR  0.00  56.0  61.0  47.0
2010-01-06  GR000016716  HELLINIKON, GR  0.01  62.0  65.0  60.0
...                 ...             ...   ...   ...   ...   ...
2019-12-26  GR000016716  HELLINIKON, GR  0.00  51.0  56.0  48.0
2019-12-27  GR000016716  HELLINIKON, GR  0.00  50.0  56.0  43.0
2019-12-28  GR000016716  HELLINIKON, GR  0.00  47.0  51.0   NaN
2019-12-29  GR000016716  HELLINIKON, GR  0.69  43.0  46.0  39.0
2019-12-30  GR000016716  HELLINIKON, GR  0.18  40.0  46.0   NaN

[2797 rows x 6 columns]
            TAVG  TMAX  TMIN  HUMIDITY  PRCP  Unnamed: 6  Unnamed: 7  \
DATE                                                                   

Chech for TAVG equals to zero

In [127]:
print("RAW_DATA NAN VALUES IN TAVG \n",raw_data.TAVG.isna().sum())
print("RAW_DATA NAN VALUES IN TMAX \n",raw_data.TMAX.isna().sum())
print("RAW_DATA NAN VALUES IN TMIN \n",raw_data.TMIN.isna().sum())
print("RAW_DATA NAN VALUES IN PRCP \n",raw_data.PRCP.isna().sum())

# (raw_data.TAVG == float(0)).sum()

RAW_DATA NAN VALUES IN TAVG 
 2311
RAW_DATA NAN VALUES IN TMAX 
 944
RAW_DATA NAN VALUES IN TMIN 
 833
RAW_DATA NAN VALUES IN PRCP 
 478


In [128]:
print("RAW_DATA2 NAN VALUES IN TAVG \n",raw_data2.TAVG.isna().sum())
print("RAW_DATA2 NAN VALUES IN TMAX \n",raw_data2.TMAX.isna().sum())
print("RAW_DATA2 NAN VALUES IN TMIN \n",raw_data2.TMIN.isna().sum())
print("RAW_DATA2 NAN VALUES IN PRCP \n",raw_data2.PRCP.isna().sum())

# (raw_data.TAVG == float(0)).sum()

RAW_DATA2 NAN VALUES IN TAVG 
 0
RAW_DATA2 NAN VALUES IN TMAX 
 0
RAW_DATA2 NAN VALUES IN TMIN 
 0
RAW_DATA2 NAN VALUES IN PRCP 
 0


Stack Overflow 
<code> data_query = data[(data['year'] >= 2005) & (data['year'] <= 2010)] </code>

TAVG_Outliers contains rows that TAVG.isna() AND TMAX OR TMIN are .notna()

THIS WORKS

In [129]:
raw_data.TAVG.fillna(0,inplace=True)
raw_data.TMAX.fillna(0,inplace=True)    
raw_data.TMIN.fillna(0,inplace=True)
raw_data.PRCP.fillna(0,inplace=True)

raw_data.TAVG.mask( ((raw_data.TAVG > raw_data.TMAX) | (raw_data.TAVG < raw_data.TMIN)) &  ((raw_data.TMAX != 0) & (raw_data.TMIN!=0) & (raw_data.TAVG == 0)), (raw_data.TMAX +raw_data.TMIN) /2 ,inplace =True)

# raw_data.to_string()
(raw_data.TAVG == 0).sum()

raw_data.loc[raw_data.TAVG == 0]


,STATION,NAME,PRCP,TAVG,TMAX,TMIN
DATE,,,,,,
1957-03-09,GR000016716,"HELLINIKON, GR",0.01,0.0,62.0,0.0


Farencheit to Celcius


In [130]:
raw_data.TAVG.mask(raw_data.TAVG > 0 ,((raw_data.TAVG - 32.0) * 5.0 / 9.0) , inplace=True)
raw_data.TAVG

DATE
1955-01-01    14.166667
1955-01-02    10.833333
1955-01-03    12.500000
1955-01-04    13.055556
1955-01-05    13.055556
                ...    
2020-12-27    16.666667
2020-12-28    15.000000
2020-12-29    16.111111
2020-12-30    16.666667
2020-12-31    15.555556
Name: TAVG, Length: 23251, dtype: float64

In [131]:
raw_data2.PRCP.mask( raw_data2.PRCP >0 , raw_data2.PRCP/100 , inplace=True)
raw_data2

,TAVG,TMAX,TMIN,HUMIDITY,PRCP,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
DATE,,,,,,,,,,,,,,,,,,,,,
2010-01-01,17.9,18.1,17.8,61.4,0.91,33,1003.6,1006.3,1002.0,0.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-02,15.6,15.7,15.5,57.4,0.70,45,1005.2,1008.7,1001.5,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-03,13.5,13.6,13.4,56.0,0.76,39,1011.7,1016.7,1008.6,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,9.5,9.6,9.5,50.7,0.60,38,1021.3,1023.1,1016.8,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,13.4,13.5,13.4,70.5,0.82,54,1018.7,1022.1,1015.5,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,10.1,10.2,10.0,60.3,0.79,44,1018.4,1019.9,1016.8,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-28,8.3,8.4,8.2,60.9,0.82,46,1016.0,1017.2,1014.2,7.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,6.4,6.5,6.4,73.4,0.82,66,1017.6,1018.9,1016.5,3.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
